In [ ]:
import pandas as pd
from azure.storage.blob import BlobServiceClient
import io

# Configuration
account_name = "retailstorage11"
account_key = "Key of Azure Account"
container_name = "retail"

print("🔄 Attempting to connect to Azure Blob Storage...")

try:
    # Create blob service client
    account_url = f"https://{account_name}.blob.core.windows.net"
    print(f"📍 Account URL: {account_url}")
    
    blob_service_client = BlobServiceClient(account_url=account_url, credential=account_key)
    
    # Test connection by getting container client
    container_client = blob_service_client.get_container_client(container_name)
    print(f"📂 Container: {container_name}")
    
    # List blobs in container
    print("🔍 Listing blobs...")
    blob_list = list(container_client.list_blobs())
    
    print(f"✅ Successfully connected! Found {len(blob_list)} files:")
    
    if len(blob_list) == 0:
        print("📭 No files found in the container")
    else:
        for i, blob in enumerate(blob_list[:10]):  # Show first 10 files
            print(f"  {i+1}. {blob.name} ({blob.size} bytes)")
        
        if len(blob_list) > 10:
            print(f"  ... and {len(blob_list) - 10} more files")

except Exception as e:
    print(f"❌ Error: {e}")
    print(f"Error type: {type(e).__name__}")


In [ ]:
# Create Gold layer aggregations - Business Intelligence metrics
# This creates daily sales summaries by product and store location

gold_df = silver_df.groupby([
    "transaction_date",     # Group by each day
    "product_id", 
    "product_name", 
    "category",            # Product category (e.g., Electronics, Clothing)
    "store_id",            # Fixed typo: was "store id" 
    "store_name", 
    "location"             # Store location/city
]).agg({
    "quantity": "sum",                    # Total units sold per product per store per day
    "total_amount": ["sum", "mean"],      # Total revenue + average transaction value
    "transaction_id": "nunique"           # Count of unique transactions
}).reset_index()

# Flatten column names after aggregation
gold_df.columns = [
    "transaction_date", "product_id", "product_name", "category", 
    "store_id", "store_name", "location",
    "total_quantity_sold",          # How many units sold
    "total_sales_amount",           # Total revenue generated  
    "average_transaction_value",    # Average $ per transaction
    "number_of_transactions"        # How many separate purchases
]

print("🏆 Gold Layer - Business Intelligence Metrics Created!")
print("=" * 60)
print(f"Shape: {gold_df.shape}")
print(gold_df.head())

In [ ]:
# Save Gold layer data to Azure Blob Storage
gold_blob_name = "gold/business_metrics/daily_sales_summary.parquet"

try:
    print(f"💾 Saving Gold layer data...")
    
    # Convert DataFrame to parquet bytes
    parquet_buffer = io.BytesIO()
    gold_df.to_parquet(parquet_buffer, index=False)
    parquet_buffer.seek(0)
    
    # Upload to blob storage
    blob_client = container_client.get_blob_client(gold_blob_name)
    blob_client.upload_blob(parquet_buffer.getvalue(), overwrite=True)
    
    print(f"✅ Successfully saved Gold layer to {gold_blob_name}")
    print(f"   Records saved: {gold_df.shape[0]}")
    print(f"   Business metrics ready for analytics!")
    
except Exception as e:
    print(f"❌ Error saving Gold layer: {e}")

In [ ]:
# Read the Gold layer dataset we just created (pandas equivalent)
gold_blob_name = "gold/business_metrics/daily_sales_summary.parquet"

try:
    print(f"📖 Loading Gold layer dataset...")
    
    # Read the gold layer data from Azure Blob Storage
    blob_client = container_client.get_blob_client(gold_blob_name)
    blob_data = blob_client.download_blob().readall()
    
    # Create the gold dataset DataFrame (equivalent to creating table)
    retail_gold_sales_summary = pd.read_parquet(io.BytesIO(blob_data))
    
    print(f"✅ Successfully loaded Gold layer dataset")
    print(f"   Shape: {retail_gold_sales_summary.shape}")
    print(f"   Columns: {list(retail_gold_sales_summary.columns)}")
    print("\nFirst 5 rows:")
    print(retail_gold_sales_summary.head())
    
    print(f"\n🏆 Gold layer table 'retail_gold_sales_summary' is ready!")
    print("   Use this DataFrame for business analytics and reporting")
    
except Exception as e:
    print(f"❌ Error loading Gold layer dataset: {e}")

In [ ]:
# Display all data from retail_gold_sales_summary (pandas equivalent of SELECT *)
print("🏆 GOLD LAYER - Business Intelligence Summary")
print("=" * 70)

# Show basic info
print(f"📊 Dataset Overview:")
print(f"   Total Records: {retail_gold_sales_summary.shape[0]}")
print(f"   Total Columns: {retail_gold_sales_summary.shape[1]}")
print(f"   Columns: {list(retail_gold_sales_summary.columns)}")

print(f"\n📋 All Gold Layer Data:")
print("-" * 70)
# Display all records
print(retail_gold_sales_summary)

print(f"\n📈 Data Summary:")
print("-" * 30)
print(retail_gold_sales_summary.describe())

print(f"\n💰 Key Business Metrics:")
print("-" * 30)
print(f"   Total Revenue: ${retail_gold_sales_summary['total_sales_amount'].sum():,.2f}")
print(f"   Total Units Sold: {retail_gold_sales_summary['total_quantity_sold'].sum():,}")
print(f"   Total Transactions: {retail_gold_sales_summary['number_of_transactions'].sum():,}")
print(f"   Average Transaction Value: ${retail_gold_sales_summary['average_transaction_value'].mean():.2f}")

In [ ]:
# Export Gold layer data to CSV for Power BI
csv_blob_name = "gold/powerbi/retail_gold_sales_summary.csv"

try:
    print("📊 Exporting Gold layer data to CSV for Power BI...")
    
    # Convert DataFrame to CSV bytes
    csv_buffer = io.StringIO()
    retail_gold_sales_summary.to_csv(csv_buffer, index=False)
    csv_data = csv_buffer.getvalue().encode('utf-8')
    
    # Upload CSV to blob storage
    blob_client = container_client.get_blob_client(csv_blob_name)
    blob_client.upload_blob(csv_data, overwrite=True)
    
    print(f"✅ CSV exported to: {csv_blob_name}")
    print("📥 You can now download this CSV from Azure Storage Explorer")
    
except Exception as e:
    print(f"❌ Error exporting CSV: {e}")